In [1]:
# import dependencies
import pandas as pd
import gmaps.geojson_geometries
import gmaps.datasets
from config import (gkey)
import numpy as np
import matplotlib.pyplot as plt
from us import states
from scipy.stats import linregress

In [2]:
gmaps.configure(api_key=gkey)

In [3]:
gmaps.geojson_geometries.list_geometries()
state_geojson = gmaps.geojson_geometries.load_geometry('us-states')

fig = gmaps.figure()

geojson_layer = gmaps.geojson_layer(state_geojson)
fig.add_layer(geojson_layer)

In [4]:
election = pd.read_csv('Resources/election.csv')
election['state_name'] = ''
election.head()

,state,trump-clinton,clinton,trump,state_name
0,AK,0.147,0.366,0.513,
1,AL,0.277,0.344,0.621,
2,AR,0.269,0.337,0.606,
3,AZ,0.036,0.451,0.487,
4,CA,-0.301,0.617,0.316,


In [6]:
abb = dict(eval(open('Resources/us_state_abbrev.py').read()))
for index, row in election.iterrows():  
    state_name = [key for key, value in abb.items() if value == row['state']]
    election.loc[index, 'state_name'] = state_name
election = election.drop(columns = ['state']).rename(columns={"state_name": "State"})
election.head()

,trump-clinton,clinton,trump,State
0,0.147,0.366,0.513,Alaska
1,0.277,0.344,0.621,Alabama
2,0.269,0.337,0.606,Arkansas
3,0.036,0.451,0.487,Arizona
4,-0.301,0.617,0.316,California


In [21]:
# run through geoJSON state list and append colors representing political party
colors = []
for feature in state_geojson['features']:
    state_name = feature['properties']['NAME']
    for index, row in election.iterrows():
        if (state_name == row['State']) & (row['trump-clinton'] >= 0):
            color= (255, 0, 0 , .2)
        elif (state_name == row['State']) & (row['trump-clinton'] < 0):
            color= (0, 0, 255 , .2)
    colors.append(color)  

In [22]:
# check length of colors
len(colors)

52

In [23]:
fig = gmaps.figure()
gini_layer = gmaps.geojson_layer(
    state_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=.4)
fig.add_layer(gini_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
normalised_shootingsbystate_df = pd.read_csv("Resources/normalised_shootingdata_bystate.csv")
normalised_shootingsbystate_df.head()

,State,Total_Shootings,Unarmed_Shootings_%,Black_Victims_%,Unarmed_Black_Victims_%,White_Victims_%,Unarmed_White_Victims_%,Hispanic_Victims_%,Unarmed_Hispanic_Victims_%,Latitude,Longitude
0,Alabama,13.925395,23.926316,35.997368,27.573684,63.873684,12.463158,2.514309,1.000000,32.7794,-86.8287
1,Alaska,5.545194,46.375000,9.937500,1.000000,58.291667,46.375000,1.000000,1.000000,64.0685,-152.2782
2,Arizona,31.964132,40.243243,9.212838,18.057432,45.408784,20.621622,54.785262,29.986486,34.2744,-111.6602
3,Arkansas,10.800574,23.376712,39.198630,35.582192,60.332192,8.458904,1.000000,1.000000,34.8938,-92.4426
4,California,100.000000,45.274608,19.665478,24.408345,26.562054,11.874465,65.849590,50.570613,37.1841,-119.4696


In [25]:
locations = normalised_shootingsbystate_df[["Latitude", "Longitude"]].astype(float)
total_shootings = normalised_shootingsbystate_df["Total_Shootings"].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=total_shootings, 
                                 dissipating=False, max_intensity=50,
                                 point_radius = 3)
fig.add_layer(gini_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
blackvictim_shootings = normalised_shootingsbystate_df["Black_Victims_%"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=blackvictim_shootings, 
                                 dissipating=False, max_intensity=75,
                                 point_radius = 3)
fig.add_layer(gini_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
whitevictim_shootings = normalised_shootingsbystate_df["White_Victims_%"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=whitevictim_shootings, 
                                 dissipating=False, max_intensity=75,
                                 point_radius = 3)
fig.add_layer(heat_layer)
fig.add_layer(gini_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
hispanicvictim_shootings = normalised_shootingsbystate_df["Hispanic_Victims_%"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=hispanicvictim_shootings, 
                                 dissipating=False, max_intensity=75,
                                 point_radius = 3)
fig.add_layer(heat_layer)
fig.add_layer(gini_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
shootingsbystate_df = pd.read_csv("Resources/shootingdata_bystate.csv")
shootingsbystate_df.head()

In [ ]:
election_shootingbystate = pd.merge(shootingsbystate_df, election, how="left", on="State").fillna(0)
election_shootingbystate["trump"] = election_shootingbystate["trump"]*100
election_shootingbystate.head()